In [1]:
import matplotlib.pyplot as plt
from nilearn.glm.first_level import FirstLevelModel
from nilearn import image, plotting
import pandas as pd
from __future__ import division
import shutil as sh
from os.path import join 
from glob import glob
import itertools
from distutils.dir_util import copy_tree
import shutil as shutil
from glob import glob
import pandas as pd
import numpy as np
from subprocess import check_call
import matplotlib.pyplot as plt
#from utilities import ensure_dir, get_n_vols
from pylab import plot, ylim, xlim, show, xlabel, ylabel, grid
from numpy import linspace, loadtxt, ones, convolve
from nipype.interfaces.dcm2nii import Dcm2niix, Dcm2nii
#import nipype_init_new
#nipype_init_new.init()
from nilearn.connectome import sym_matrix_to_vec, vec_to_sym_matrix,ConnectivityMeasure
from nilearn.plotting import plot_stat_map, plot_anat, plot_img, view_img, plot_matrix, plot_glass_brain
from nilearn.image import concat_imgs, mean_img, load_img, index_img, math_img, resample_img
from nilearn.masking import apply_mask
from nilearn.plotting import plot_stat_map, plot_anat, plot_img, view_img, plot_design_matrix
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn import plotting
from statsmodels.sandbox.stats.multicomp import multipletests as multi
from scipy.stats import norm
import statsmodels.stats.multitest as ssm
import statsmodels
import seaborn as sns
from scipy.stats import pearsonr,ttest_ind
from sklearn.model_selection import StratifiedKFold,permutation_test_score
from sklearn.svm import SVC
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVR
import time

/home/haiyanwu/miniconda3/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "
/home/haiyanwu/miniconda3/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def pad_vector(contrast_, n_columns):
    """A small routine to append zeros in contrast vectors"""
    return np.hstack((contrast_, np.zeros(n_columns - len(contrast_))))
def ensure_dir(path):
    d = os.path.dirname(path)
    if not os.path.exists(d):
        os.makedirs(d)
def get_n_vols(iname):
    img = load_img(iname)
    dims=np.shape(image.get_data(img))
    return 1 if len(dims) < 4 else dims[3]
def compute_contrast(design_matrices,contrast_name):
    dmcols=[design_matrices.columns.values.tolist()]
    for i in range(len(dmcols)):
        for j in range(len(dmcols[i])):
            if contrast_name in str(dmcols[i][j]):
                dmcols[i][j]=1
            else:
                dmcols[i][j]=0
    return dmcols
def list_substract(a,b):
    new_list=[]
    for i in range(len(a)):
        new_list.append([])
        for j in range(len(a[i])):
            new_list[i].append(0)
            new_list[i][j]=a[i][j]-b[i][j]
    return new_list

In [11]:
base_dir='/media/haiyanwu/HDD4/fmri2019/'
beta_dir=base_dir+'2.GLM/3.GLM_SPM/results/GLM11/betamap/'
ensure_dir(beta_dir)


In [12]:
subList=[102,103,105,107,108,110,111,112,113,114,115,116,117,118,119,120,121,122,123,125,127,128,129,130,131,132]

In [68]:
# considering some over time trials were removed so the trial number of each session was not the same, we didn't combin four sessions together to run the GLM 

In [8]:

tr=2

for subno in subList:

    subdir_beta=beta_dir+"sub-%03d/"%subno
    ensure_dir(subdir_beta)
    
    
    for sesno in [1,2,3,4]:
        ## Prepare data and analysis parameters
        # load functional images
        fd=load_img(base_dir+"fmriprep-deriv/fmriprep/sub-%03d/ses-%01d/func/sub-%03d_ses-%01d_preproc_only_masked.nii.gz"%(subno,sesno,subno,sesno))
        n_scans=get_n_vols(fd)

        # load the events file
        events=pd.read_csv(base_dir+"2.GLM/event_files_noDelete/sub-%03d/sub-%03d_ses-%01d_task-ua%01d_bold.tsv"%(subno,subno,sesno,sesno), sep="\t")
        events=events.rename(columns={'time_onset_new':'onset'})
        events['duration']=0
        events=events.reset_index(drop=True)
        events['item']=list(map(lambda x:'item%d'%(x),events['menu']))
        
        # load the confound file
        confounds=pd.read_table(base_dir+"2.GLM/confound_files/sub-%03d/ses-%01d/sub-%03d_ses-%01d_confound.txt"%(subno,sesno,subno,sesno),header=None,sep=',')

        dm_all=[]

        for i_trial in range(len(events)):
            evs=events
            evs.fillna(4,inplace=True)
            evs['trial_type']=evs['condition']
            evs.loc[i_trial,'trial_type']=evs.loc[i_trial,'item']
            evs=evs[['onset','trial_type','duration']]
            frame_times=np.arange(n_scans) * tr
            dm=make_first_level_design_matrix(
                    frame_times=frame_times, events=evs, drift_model='cosine', high_pass=1/128,
                    add_regs=confounds, hrf_model='spm',min_onset=0)
            dm_all.append(dm)
    
   
            glm_LSS = FirstLevelModel(t_r=tr,
                                slice_time_ref=1, # default is 0, but in SPM default is 1; used the middle slice
                                hrf_model='spm + derivative', # block design: spm; event-related: spm + derivative
                                noise_model='ar1',
                                drift_model='cosine',
                                high_pass=1/128,
                                standardize=False,
                                min_onset=0,
                                mask_img=False,
                                smoothing_fwhm=None,
                                verbose=0
                                )
            starttime = time.time()
            glm_LSS = glm_LSS.fit(fd,design_matrices=dm)
            print("- %f"%(time.time()-starttime))


            basic_contrasts=dict()
            # Generate the contrasts
            for cname in [evs.loc[i_trial,'trial_type']]:
                temp_name=cname
                temp_contrast=compute_contrast(dm,cname)
                basic_contrasts[temp_name]=temp_contrast




            from nilearn import plotting
            # Iterate on contrasts
            for index, (contrast_id, contrast_val) in enumerate(basic_contrasts.items()):
                
                ## estimate the contasts
                ## note that the model implictly compute a fixed effects across the two sessions
                ## effect size
                eff_map = glm_LSS.compute_contrast(
                    contrast_val, output_type='effect_size')
                t_map = glm_LSS.compute_contrast(
                    contrast_val, stat_type='t', output_type='stat')


                eff_map.to_filename(join(subdir_beta, 'sub-%03d_%s_eff_map.nii.gz'%(subno,events.loc[i_trial,'trial_type'])))
                t_map.to_filename(join(subdir_t, 'sub-%03d_%s_t_map.nii'%(subno,events.loc[i_trial,'trial_type'])))

- 15.448073
- 4.839701
- 4.957064
- 4.870112
- 5.693078
- 6.070691
- 7.632357
- 13.742362
- 15.455689
- 5.862481
- 4.762331
- 7.674881
- 4.769329
- 4.634922
- 4.665871
- 5.231818
- 12.363892
- 5.417835
- 5.237642
- 4.756443
- 5.195347
- 4.736165
- 4.626699
- 4.664082
- 5.336468
- 12.040671
- 4.700148
- 5.029518
- 4.738018
- 5.190962
- 4.734983
- 4.636875
- 4.690842
- 5.109178
- 11.638458
- 5.615868
- 5.100887
- 4.814097
- 5.254366
- 4.919936
- 4.771035
- 4.863922
- 6.035198
- 10.997987
- 4.964586
- 4.943069
- 4.751654
- 5.104729
- 4.650279
- 4.704206
- 5.280163
- 5.520832
- 10.571774
- 4.944957
- 4.899396
- 4.651838
- 5.121228
- 4.622845
- 4.622794
- 5.290939
- 6.199885
- 6.145261
- 4.545722
- 5.074968
- 4.740953
- 5.163201
- 4.632609
- 4.622281
- 4.672607
- 5.366320
- 4.925130
- 5.089643
- 4.788826
- 4.985965
- 4.611107
- 5.037679
- 4.613982
- 4.613662
- 4.660041
- 5.823739
- 6.336954
- 5.126042
- 5.218621
- 4.655433
- 4.898093
- 4.733775
- 4.606527
- 4.597550
- 5.106544
- 5.272506
- 

In [71]:
fname='/media/haiyanwu/HDD4/fmri2019/2.GLM/3.GLM_SPM/results/GLM5/betamap/sub-103/sub-103_ses1_trial1_eff_map.nii.gz'
nib.load(fname).get_fdata()

array([[[7.16005384e-14, 7.16005384e-14, 7.16005384e-14, ...,
         7.16005384e-14, 7.16005384e-14, 7.16005384e-14],
        [7.16005384e-14, 7.16005384e-14, 7.16005384e-14, ...,
         7.16005384e-14, 7.16005384e-14, 7.16005384e-14],
        [7.16005384e-14, 7.16005384e-14, 7.16005384e-14, ...,
         7.16005384e-14, 7.16005384e-14, 7.16005384e-14],
        ...,
        [7.16005384e-14, 7.16005384e-14, 7.16005384e-14, ...,
         7.16005384e-14, 7.16005384e-14, 7.16005384e-14],
        [7.16005384e-14, 7.16005384e-14, 7.16005384e-14, ...,
         7.16005384e-14, 7.16005384e-14, 7.16005384e-14],
        [7.16005384e-14, 7.16005384e-14, 7.16005384e-14, ...,
         7.16005384e-14, 7.16005384e-14, 7.16005384e-14]],

       [[7.16005384e-14, 7.16005384e-14, 7.16005384e-14, ...,
         7.16005384e-14, 7.16005384e-14, 7.16005384e-14],
        [7.16005384e-14, 7.16005384e-14, 7.16005384e-14, ...,
         7.16005384e-14, 7.16005384e-14, 7.16005384e-14],
        [7.16005384e-14, 